In [1]:
import scanpy as sc
import anndata as an
import pandas as pd
import numpy as np
# import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams["font.family"] = "Arial"

sc.set_figure_params(figsize=(4, 4))

palette = ['#199de5', '#fbbc04']

In [2]:

sc.set_figure_params(facecolor="white", figsize=(8, 8), dpi_save=300)
sc.settings.verbosity = 1
sc.settings.figdir = './figures-cell2location/'

In [3]:
Vsamples = {'A13':['152806', '152810'],
            'A30':['152807', '152811'],
            'donorX_lesion':['UA_Endo13041816', 'UA_Endo13041817'],
            'donorY_lesion':['HCA_ENDO_sp13458406', 'HCA_ENDO_sp13458407'],
            'FX1101': ['UA_HUTER_sp_10879892'],
            'FX0033':['UA_HUTER_sp_10879893'],
            'FX0028':['UA_HUTER_sp_10879894'],
            'FX0060':['UA_HUTER_sp_10879895']
           }
path = '/nfs/team292/lg18/endometriosis/data/visium/'




# Reading Visium data


The function read_visium returns an AnnData object that contains counts, images and spatial coordinates. We will calculate standards QC metrics with pp.calculate_qc_metrics and visualize them.

An anndata will be saved for cell2location.


In [4]:
def read_and_qc(sample_name, path):

    adata = sc.read_visium(path + str(sample_name),
                           count_file='filtered_feature_bc_matrix.h5', load_images=True)
    adata.obs['sample'] = sample_name
    adata.var['SYMBOL'] = adata.var_names
    adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    adata.var_names = adata.var['ENSEMBL']
    adata.var.drop(columns='ENSEMBL', inplace=True)

    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.var['mt'] = [gene.startswith('mt-') for gene in adata.var['SYMBOL']]
    adata.obs['mt_frac'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']

    # mitochondria-encoded (MT) genes should be removed for spatial mapping
    adata.obsm['mt'] = adata[:, adata.var['mt'].values].X.toarray()
    adata = adata[:, ~adata.var['mt'].values]

    # add sample name to obs names
    adata.obs["sample"] = [str(i) for i in adata.obs['sample']]
    adata.obs_names = adata.obs["sample"] \
                          + '_' + adata.obs_names
    adata.obs.index.name = 'spot_id'
#     adata.obs['barcode_sample'] = adata.obs_names

    oldname = list(adata.uns['spatial'].keys())[0]
    adata.uns['spatial'][sample_name] = adata.uns['spatial'].pop(oldname)


    return adata


def select_slide(adata, s, s_col='sample'):
    r""" Select data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial samples
    :param s: name of selected sample
    :param s_col: column in adata.obs listing sample name for each location
    """

    slide = adata[adata.obs[s_col].isin([s]), :]
    s_keys = list(slide.uns['spatial'].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns['spatial'] = {s_spatial: slide.uns['spatial'][s_spatial]}

    return slide

In [5]:
for sam in Vsamples.keys():
    print(sam)
    # read first sample
    adata = read_and_qc(Vsamples[sam][0], path=path+'/')

    # read the remaining samples
    slides = {}
    for i in Vsamples[sam][1:]:
        adata_1 = read_and_qc(i, path=path+'/')
        slides[str(i)] = adata_1

    adata_0 = adata.copy()

    # combine individual samples
    adata = adata.concatenate(
        list(slides.values()),
        batch_key="sample",
        uns_merge="unique",
        batch_categories=Vsamples[sam],
        index_unique=None
    )
    # merging metadata
    print('merging')
    adata.obs['sample_name'] = sam

    
    os.system('mkdir -p '+ path+'/cell2location/'+ sam +'/') 
    adata.write(path + '/cell2location/' + sam + '/'+ sam + '_RNA_visium.h5ad')
      

A13


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
... storing 'sample_name' as categorical
... storing 'feature_types' as categorical
... storing 'genome' as categorical


merging


... storing 'SYMBOL' as categorical


A30


...

... storing 'SYMBOL' as categorical


FX0060


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.
... storing 'in_tissue' as categorical
... storing 'array_row' as categorical
... storing 'array_col' as categorical
... storing 'sample_name' as categorical
... storing 'feature_types' as categorical
... storing 'genome' as categorical


merging


... storing 'SYMBOL' as categorical


# Reading the merged scRNA dataset


In [6]:
path2sc = '/nfs/team292/lg18/endometriosis/integrated_objects/'
adata = sc.read(path2sc+'cells_integrated.h5ad')
adata.X.shape

(390867, 17736)

In [7]:
## All celltypes
annot_df = pd.read_csv('/nfs/team292/lg18/endometriosis/annotations/cells_endometrium.csv',index_col=0)

cell_annot = annot_df['Mareckova_celltype'].to_dict()
adata.obs['Mareckova_celltype'] = adata.obs_names.map(cell_annot)
print(adata.obs['Mareckova_celltype'].value_counts())

/home/jovyan/my-conda-envs/cellpymc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (5,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,39,40,41,42,46,47,55,56,58,59,60,62,69) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Mesen_dStromal          96529
Mesen_eStromal          79488
Immune_Lymphoid         34456
Epi_SOX9                32015
Epi_Glandular           27887
Endothelial             23519
Epi_preGlandular        16765
Epi_Luminal             14253
Epi_Horm                11490
Mesen_ePV               11325
Immune_Myeloid          11260
LowQC                    7549
Epi_Ciliated             7467
Doublet                  5616
Mesen_mPV                4618
Mesen_uSMCs              2995
Epi_endocervix_MUC5B     1748
Epi_Glands                952
Mesen_FibBasalis          539
Epi_cervix_KRT5           396
Name: Mareckova_celltype, dtype: int64


### Add cell type annotations from subanalysis as LABELS

In [8]:
# MESENCHYMAL
annot_df = pd.read_csv('/nfs/team292/lg18/endometriosis/annotations/cells_endometrium_mesenchymal_CLEAN.csv',index_col=0)
annot_df.at[annot_df['Mareckova_mesen_celltype'] == 'ePV_AOC3', 'Mareckova_mesen_lineage'] = 'ePV_AOC3'
annot_df.at[annot_df['Mareckova_mesen_celltype'] == 'ePV_MMP11', 'Mareckova_mesen_lineage'] = 'ePV_MMP11'
annot_df.at[annot_df['Mareckova_mesen_celltype'] == 'ePV_STEAP4', 'Mareckova_mesen_lineage'] = 'ePV_STEAP4'
annot_df.at[annot_df['Mareckova_mesen_celltype'] == 'dStromal_early', 'Mareckova_mesen_lineage'] = 'dStromal_early'
annot_df.at[annot_df['Mareckova_mesen_celltype'] == 'eStromal_MMPs', 'Mareckova_mesen_lineage'] = 'eStromal_MMPs'
annot_df.at[annot_df['Mareckova_mesen_celltype'] == 'Fibroblast_basalis', 'Mareckova_mesen_lineage'] = 'Fibroblast_basalis'


annot_df['Mareckova_mesen_celltype'] = [ 'Mesenchymal '+ i for i in annot_df['Mareckova_mesen_celltype']]
annot_df['Mareckova_mesen_lineage'] = [ 'Mesenchymal '+ i for i in annot_df['Mareckova_mesen_lineage']]
annot1 = annot_df['Mareckova_mesen_lineage'].to_dict()
annot11 = annot_df['Mareckova_mesen_celltype'].to_dict()


# EPITHELIAL
annot_df = pd.read_csv('/nfs/team292/lg18/endometriosis/annotations/cells_endometrium_epithelial_CLEAN.csv',index_col=0)
annot_df.at[annot_df['Mareckova_epi_celltype'] == 'SOX9_basalis_I', 'Mareckova_epi_lineage'] = 'SOX9_basalis_I'
annot_df.at[annot_df['Mareckova_epi_celltype'] == 'SOX9_basalis_II', 'Mareckova_epi_lineage'] = 'SOX9_basalis_II'
annot_df.at[annot_df['Mareckova_epi_celltype'] == 'SOX9_luminal', 'Mareckova_epi_lineage'] = 'SOX9_luminal'
annot_df.at[annot_df['Mareckova_epi_celltype'] == 'Glandular_secretory', 'Mareckova_epi_lineage'] = 'Glandular_secretory'
annot_df.at[annot_df['Mareckova_epi_celltype'] == 'Glandular_secretory_FGF7', 'Mareckova_epi_lineage'] = 'Glandular_secretory'
annot_df.at[annot_df['Mareckova_epi_celltype'] == 'preLuminal', 'Mareckova_epi_lineage'] = 'preLuminal'

annot_df['Mareckova_epi_celltype'] = [ 'Epithelial '+ i for i in annot_df['Mareckova_epi_celltype']]
annot_df['Mareckova_epi_lineage'] = [ 'Epithelial '+ i for i in annot_df['Mareckova_epi_lineage']]
annot2 = annot_df['Mareckova_epi_lineage'].to_dict()
annot22 = annot_df['Mareckova_epi_celltype'].to_dict()


# add to andata
annot_all = adata.obs['Mareckova_celltype'].to_dict()
annot = dict(annot_all)
annot.update(annot1)
annot.update(annot2)

annotF = dict(annot_all)
annotF.update(annot11)
annotF.update(annot22)

adata.obs['Mareckova_fine_lineage'] = adata.obs_names.map(annot)
print(adata.obs['Mareckova_fine_lineage'].value_counts())

adata.obs['Mareckova_fine_celltype'] = adata.obs_names.map(annotF)
print(adata.obs['Mareckova_fine_celltype'].value_counts())
# set(adata.obs['Mareckova_fine_celltype'])

Mesenchymal eStromal              63353
Mesenchymal dStromal              56171
Immune_Lymphoid                   34456
Endothelial                       23519
Mesen_dStromal                    22381
Mesenchymal dStromal_early        16804
Epithelial Hormones               14381
Epithelial Glandular              13050
Epithelial SOX9                   12583
Epithelial preGlandular           11840
Immune_Myeloid                    11260
Epithelial Luminal                10250
Epithelial Glandular_secretory     9414
Epi_SOX9                           8638
Mesen_eStromal                     7902
LowQC                              7549
Mesenchymal ePV_MMP11              7012
Epi_Glandular                      6604
Epithelial Ciliated                5738
Doublet                            5616
Mesenchymal Hormones               4132
Epithelial Cycling                 4129
Mesenchymal eStromal_MMPs          3993
Epi_Luminal                        3005
Mesenchymal ePV_AOC3               2754


In [9]:
# Rename epithalial ciliated,  which are not included in the zoom in
adata.obs.at[adata.obs['Mareckova_fine_lineage'] == 'Epithelial_Ciliated', 'Mareckova_fine_lineage'] = 'Epithelial Ciliated'

# Remove unreliable populations
adata = adata[[ 'Doublet' not in i for i in adata.obs['Mareckova_fine_lineage'] ]]
adata = adata[[ 'LowQC' not in i for i in adata.obs['Mareckova_fine_lineage'] ]]
adata = adata[[ 'Epi_' not in i for i in adata.obs['Mareckova_fine_lineage'] ]]
adata = adata[[ 'Mesen_' not in i for i in adata.obs['Mareckova_fine_lineage'] ]]
adata = adata[[ 'unknown' not in i for i in adata.obs.Mareckova_fine_celltype]]
adata = adata[[ 'Unknown' not in i for i in adata.obs['Mareckova_fine_lineage'] ]]
adata = adata[[ 'Hormones' not in i for i in adata.obs['Mareckova_fine_lineage'] ]]
adata = adata[[ 'Other' not in i for i in adata.obs.Mareckova_fine_celltype]]
adata = adata[[ i not in ['Immune'] for i in adata.obs['Mareckova_fine_lineage'] ]]
print(adata.obs['Mareckova_fine_lineage'].value_counts())

/home/jovyan/my-conda-envs/cellpymc/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


Mesenchymal eStromal              63353
Mesenchymal dStromal              56171
Immune_Lymphoid                   34456
Endothelial                       23519
Mesenchymal dStromal_early        16804
Epithelial Glandular              13050
Epithelial SOX9                   12583
Epithelial preGlandular           11840
Immune_Myeloid                    11260
Epithelial Luminal                10250
Epithelial Glandular_secretory     9414
Mesenchymal ePV_MMP11              7012
Epithelial Ciliated                5738
Epithelial Cycling                 4129
Mesenchymal eStromal_MMPs          3993
Mesenchymal ePV_AOC3               2754
Epithelial preLuminal              2735
Epithelial SOX9_luminal            2700
Epithelial SOX9_basalis_II         1590
Epithelial MUC5B                   1389
Mesenchymal uSMCs                  1386
Mesenchymal ePV_STEAP4             1382
Mesenchymal mPV                    1354
Mesenchymal HOXA13                  844
Epithelial KRT5                     484


In [10]:
## Remove cycling populations
adata = adata[[ 'G1' in i for i in adata.obs.phase]]
adata = adata[[ 'Cycling' not in i for i in adata.obs.Mareckova_fine_celltype]]
print(adata.obs['Mareckova_fine_lineage'].value_counts())
# print(adata.obs['Mareckova_fine_celltype'].value_counts())

Mesenchymal eStromal              55760
Mesenchymal dStromal              55649
Immune_Lymphoid                   31683
Endothelial                       21859
Mesenchymal dStromal_early        16751
Epithelial Glandular              12827
Epithelial SOX9                   12261
Epithelial preGlandular           11663
Immune_Myeloid                    10719
Epithelial Luminal                10129
Epithelial Glandular_secretory     9207
Mesenchymal ePV_MMP11              6648
Epithelial Ciliated                4394
Mesenchymal eStromal_MMPs          3987
Epithelial preLuminal              2655
Mesenchymal ePV_AOC3               2411
Epithelial SOX9_luminal            2210
Epithelial SOX9_basalis_II         1569
Mesenchymal ePV_STEAP4             1375
Mesenchymal uSMCs                  1367
Epithelial MUC5B                   1365
Mesenchymal mPV                    1352
Mesenchymal HOXA13                  837
Epithelial KRT5                     475
Mesenchymal Fibroblast_basalis      416


### Select cells with > 2000 genes expressed

In [11]:
sc.pp.filter_cells(adata, min_genes=2000)
print(adata.obs['Mareckova_fine_lineage'].value_counts())

Trying to set attribute `.obs` of view, copying.


Mesenchymal eStromal              47712
Mesenchymal dStromal              37862
Mesenchymal dStromal_early        14242
Endothelial                       13971
Epithelial preGlandular           11205
Epithelial SOX9                   11112
Epithelial Glandular               9939
Immune_Myeloid                     7998
Epithelial Glandular_secretory     7594
Immune_Lymphoid                    6409
Epithelial Luminal                 6369
Mesenchymal ePV_MMP11              5626
Epithelial Ciliated                3833
Mesenchymal eStromal_MMPs          3799
Epithelial preLuminal              2515
Epithelial SOX9_luminal            2184
Mesenchymal ePV_AOC3               2015
Epithelial SOX9_basalis_II         1543
Epithelial MUC5B                   1245
Mesenchymal ePV_STEAP4              932
Mesenchymal HOXA13                  645
Mesenchymal mPV                     641
Mesenchymal uSMCs                   544
Epithelial KRT5                     434
Epithelial SOX9_basalis_I           347


### Use Ensembl id as GENE 

In [12]:
adata.var['SYMBOL'] = adata.var_names
adata.var.rename(columns={'gene_ids-0-GarciaAlonso': 'ENSEMBL'}, inplace=True)
adata.var_names = adata.var['ENSEMBL']
adata.var.drop(columns='ENSEMBL', inplace=True)

In [13]:
adata.var_names = adata.var_names.astype(str)
adata.var_names_make_unique()

In [14]:
adata.write(path + '/cell2location/scRNAseq.h5ad')

... storing 'Mareckova_celltype' as categorical
... storing 'Mareckova_fine_lineage' as categorical
... storing 'Mareckova_fine_celltype' as categorical
